In [7]:
from IPython.display import SVG, display
import numpy as np

import sys
import io
sys.path.append('..') # uncomments and adjust if your Boxes.py copy in not in the Python path
from boxes import *
from boxes import lids
from boxes.edges import Bolts
from boxes.lids import _TopEdge


class MyUniversalBox(_TopEdge):
    """Box with various options for different styles and lids"""

    ui_group = "Box"

    def __init__(self) -> None:
        Boxes.__init__(self)
        self.addTopEdgeSettings(roundedtriangle={"outset" : 1},
                                hinge={"outset" : True})
        self.addSettingsArgs(edges.FlexSettings)
        self.addSettingsArgs(lids.LidSettings)
        self.buildArgParser("top_edge", "bottom_edge",
                            "x", "y", "h", "outside")
        self.argparser.add_argument(
            "--vertical_edges",  action="store", type=str,
            default="finger joints",
            choices=("finger joints", "finger holes"),
            help="connections used for the vertical edges")

    def rect_holes(self, right=False):
        y_offset = 8
        x_offset = 15
        w, h = 2, (height_total - 2 * y_offset)
        x_step = w + 4
        n = int((self.y - 2 * x_offset) / x_step)
        for x in np.linspace(x_offset, self.y - x_offset, n):
            if right:
                x = self.y - x
            self.rectangularHole(x, y_offset, w, h, center_y=False)
    
    def top_hole(self, x, y, top_edge):
        t = self.thickness

        if top_edge == "f":
            edge = self.edges["F"]
            self.moveTo(2*t+self.burn, 2*t, 90)
        elif top_edge == "F":
            edge = self.edges["f"]
            self.moveTo(t+self.burn, 2*t, 90)
        else:
            raise ValueError("Only f and F supported")

        for l in (y, x, y, x):
            edge(l)
            if top_edge == "F": self.edge(t)
            self.corner(-90)
            if top_edge == "F": self.edge(t)

    def render(self):
        x, y, h = self.x, self.y, self.h
        t = self.thickness

        tl, tb, tr, tf = self.topEdges(self.top_edge)
        b = self.edges.get(self.bottom_edge, self.edges["F"])

        d2 = Bolts(2)
        d3 = Bolts(3)

        d2 = d3 = None

        sideedge = "F" if self.vertical_edges == "finger joints" else "h"

        if self.outside:
            self.x = x = self.adjustSize(x, sideedge, sideedge)
            self.y = y = self.adjustSize(y)
            self.h = h = self.adjustSize(h, b, self.top_edge)

        with self.saved_context():
            callback_front = lambda: self.hole(54.5, height_hex + DC55_21_hole_offset, DC55_21_radius)
            text_callback = lambda text: self.text(text, height_hex - 3, self.x/2, align="middle center", angle=-90, color=Color.ANNOTATIONS, fontsize=10)
            self.rectangularWall(x, h, [b, sideedge, tf, sideedge],
                                 ignore_widths=[1, 6],
                                 bedBolts=[d2], move="up", label="", callback=[callback_front, lambda: text_callback("ВХІД")])
            self.rectangularWall(x, h, [b, sideedge, tb, sideedge],
                                 ignore_widths=[1, 6],
                                 bedBolts=[d2], move="up", label="", callback=[callback_front, lambda: text_callback("ВИХІД")])

            if self.bottom_edge != "e":
                self.rectangularWall(x, y, "ffff", bedBolts=[d2, d3, d2, d3], move="up", label="")
            if self.top_edge in "fF":
                self.set_source_color(Color.MAGENTA)    # I don't know why this part has a different color, but RED is not a good choice because RED is used for annotations
                self.rectangularWall(x+4*t, y+4*t, callback=[
                    lambda:self.top_hole(x, y, self.top_edge)], move="up", label="top hole")
                self.set_source_color(Color.BLACK)
            self.drawLid(x, y, self.top_edge, [d2, d3])
            self.lid(x, y, self.top_edge)

        self.rectangularWall(x, h, [b, sideedge, tf, sideedge],
                             ignore_widths=[1, 6],
                             bedBolts=[d2], move="right only", label="invisible")
        self.rectangularWall(y, h, [b, "f", tl, "f"],
                             ignore_widths=[1, 6],
                             bedBolts=[d3], move="up", label="", callback=[lambda: self.rect_holes(False)])
        self.rectangularWall(y, h, [b, "f", tr, "f"],
                             ignore_widths=[1, 6],
                             bedBolts=[d3], move="up", label="", callback=[lambda: self.rect_holes(True)])

thickness = 4.0

DC55_21_radius = 12.0 / 2
height_hex = 25
DC55_21_hole_offset = 6
height_total = 44

x, y = 123.5, 89

b = MyUniversalBox()
file_format = 'dxf'
b.parseArgs(['--reference=0', f'--format={file_format}', '--burn=0.05', '--debug=0', f'--thickness={thickness}', '--top_edge=i', '--bottom_edge=F', f'--x={x}', f'--y={y}', f'--h={height_total}', '--outside=0', '--Hinge_style=flush'])
#b.parseArgs()
b.open()
b.render()
data = b.close()

with open(f'boxes_battery_pack.{file_format}', "wb") as f:
    if isinstance(data, io.BufferedReader):
        # dxf
        f.write(data.read())
    else:
        # svg
        f.write(data.getbuffer())

if file_format == "svg":
    display(SVG(data=data.getvalue()))

pstoedit: version 3.78 / DLL interface 108 (built: Nov 23 2021 - release build - g++ 11.2.0 - 64-bit) : Copyright (C) 1993 - 2021 Wolfgang Glunz
